# Import and plot models

In [28]:
%matplotlib qt

In [1]:
import sys
sys.path.append('/mnt/c/Users/luukv/Documenten/NatuurSterrkenkundeMasterProject/CodeMP/MasterProject')

import numpy as np
import matplotlib.pyplot as plt
import os
import re

from import_data import BailerJones
from model_functions import *
from functions import import_spectra

Import models and spectra

In [3]:
models = import_models('Milkyway')
spectra = import_spectra('4U1538_52')

object_name = '4U1538-52'
distance = BailerJones().loc[BailerJones().id == object_name]['r_med_photogeo'].reset_index(drop=True).at[0]

Get models in given T and log(g) interval

In [4]:
filtered_models = models_in_interval(models, T1=30000, T2=40000, log_g1=2.8, log_g2=3.2)

Plot models and spectrum

In [7]:
plot_models_and_spectrum(filtered_models, spectra, object_name, distance)

In [6]:
# Open the text file for reading
with open('/mnt/c/Users/luukv/Documenten/NatuurSterrkenkundeMasterProject/CodeMP/MasterProject/ModelFitting/Models/ob-i_25-38_line.txt/ob-i_25-38_line.txt', 'r') as file:
    # Read all lines from the file
    lines = file.readlines()

spectrum = {'WAVELENGTH': [], 'FLUX': []}
# Process the lines as needed
for line in lines:
    # Perform operations on each line
    # For example, you might split the line into values if it's a space-separated list of numbers
    values = line.split()
    # Process the values accordingly
    spectrum['WAVELENGTH'].append(float(values[0]))
    spectrum['FLUX'].append((float(values[1])))

plt.plot(spectrum['WAVELENGTH'], spectrum['FLUX'])
plt.show()


# Sav the models as .json file for quicker load in times.

In [4]:
folder_name = '/mnt/c/Users/luukv/Documenten/NatuurSterrkenkundeMasterProject/CodeMP/MasterProject/ModelFitting/Models/smc-ob-i_sed_all'

# Get the list of all filenames in the folder
file_names = [f for f in os.listdir(folder_name) if os.path.isfile(os.path.join(folder_name, f)) and f != 'modelparameters.txt']

# Define a regex pattern to match the two numbers for T and log(g) from file name
pattern = re.compile(r'(\d+)-(\d+)')

# Import all spectra and add them to a dictionary
models = {}
for file in file_names:
    # Extract temperature and log(g) from file name
    T, log_g = map(int, pattern.findall(file)[0])
    # Get spectrum
    spectrum = import_model(folder_name + '/' + file)
    # Save spectrum
    model = {}
    model['Name'] = f'T{T * 1000}logg{log_g / 10}'
    model['WAVELENGTH'] = spectrum['WAVELENGTH']
    model['FLUX'] = spectrum['FLUX']
    model['Teff'] = T * 1000
    model['log(g)'] = log_g / 10

    models[f'T{T * 1000}logg{log_g / 10}'] = model

# Save the models as json file
with open(folder_name + '_save.json', 'w') as json_file:
    json.dump(models, json_file)

In [6]:
def import_SED_models_quickload(galaxy:str)->dict:
    """
    Imports all SED models for a given galaxy

    Args:
        galaxy (str): Galaxy to import the models for (Milkyway, SMC, LMC)

    Returns:
        models (dict): Dictionary with SED models. Name: T{Teff}logg{log(g)}
    """
    
    # Path to the models
    model_path = '/mnt/c/Users/luukv/Documenten/NatuurSterrkenkundeMasterProject/CodeMP/MasterProject/ModelFitting/Models/'

    # Assign folder name corresponding to the galaxy
    if galaxy == 'Milkyway':
        folder_name = model_path + 'gal-ob-i_sed_all'
    elif galaxy == 'SMC':
        folder_name = model_path + 'smc-ob-i_sed_all'
    elif galaxy == 'LMC':
        folder_name = model_path + 'lmc-ob-i_sed_all'

    # Load the JSON file with the models
    with open(folder_name + '_save.json', 'r') as json_file:
        models = json.load(json_file)

    return models

In [7]:
models = import_SED_models_quickload('Milkyway')

In [33]:
wav = np.array(models['T15000logg3.0']['WAVELENGTH'])
flux = np.array(models['T15000logg3.0']['FLUX'])

plt.plot(10**wav, 10**flux)
plt.xlabel(r'$\lambda$ ($\AA$)')
plt.ylabel(r'log F$_{\lambda}$ (erg cm$^{-2}$ s$^{-1}$ $\AA^{-1})$ in $d=10kpc$')
# plt.xlim(left=np.log10(3100), right=np.log10(10000))
plt.xlim(0, 10000)
plt.show()